# Scraper isole dal sito Private Islands Online

Questo è uno scraper dal sito [Private Islands Online](https://www.privateislandsonline.com/).

Al momento lo scraping online è fermo, è stato fatto invece lo scraping dal file HTML renderizzato intero.

In [77]:
# base_url = "https://www.privateislandsonline.com/search?region=&diversion=&availability=sale&price_range=0%3A50000000&size_range=0%3A1000&q=&view%5B0%5D=1&order=created_at%3ADESC&page=74"

# ci sono isole fino a 75 pagine (9 isole per pagina)

In [1]:
# libraries for requesting url and scraping jsons
import requests
import json
from bs4 import BeautifulSoup

In [4]:
# scraping directly from an HTML file of the entire page with all islands
file = open('file-completo.html')
content = file.read()
#parse HTML
soup = BeautifulSoup(content, 'html.parser')

In [173]:
names = []
urls = []
acres = []
prices = []
countries = []
infos = []

In [174]:
for island in soup.select('div[class="col-sm-6 col col-md-4 col-xs-offset-1 col-sm-offset-0 col-xs-10 padd-lr-8"]'):
    try:
        r_name = island.find("span", class_="name")
        r_url = island.find("a", href=True)
        r_acres = island.find("span", class_="num")
        r_prices = island.find("div", class_="list-price")
        r_countries = island.find("div", class_="list-name")
        names.append(r_name.text)
        urls.append(r_url['href'])
        acres.append(r_acres.text)
        prices.append(r_prices.text)
        countries.append(r_countries.text)
    except:
        acres.append("Acres not available")
        prices.append("Price not available")
        countries.append("Country not available")

In [113]:
import pandas as pd

In [175]:
list_of_tuples = list(zip(names, urls, acres, prices, countries))

In [181]:
df = pd.DataFrame(list_of_tuples, columns = ['Name', 'Url', 'Size in acres', "Price in USD", "Location"])

In [182]:
df

,Name,Url,Size in acres,Price in USD,Location
0,\nRangyai Island\n,https://www.privateislandsonline.com/asia/thai...,110,"$\nUSD 160,000,000.00\n","Thailand\n, Asia"
1,\nBlackbook Listing\n(2943) \n,https://www.privateislandsonline.com/africa/se...,170 - 190,$ \nPrice Upon Request\n,"Seychelles\n, Africa"
2,\nBlackbook Listing\n(2867) \n,https://www.privateislandsonline.com/europe/sp...,100 - 200,$ \nPrice Upon Request\n,"Spain\n, Europe"
3,\nPumpkin Key\n,https://www.privateislandsonline.com/united-st...,26,"$\nUSD 95,000,000.00\n","Florida\n, United States"
4,\nSpectabilis Island\n,https://www.privateislandsonline.com/caribbean...,460,$ \nSold,"The Exumas\n, Bahamas\n, Caribbean"
...,...,...,...,...,...
666,\nSavage Island Plantation\n,https://www.privateislandsonline.com/united-st...,415,$ \nOff the Market,"South Carolina\n, United States"
667,\nComfort Island\n,https://www.privateislandsonline.com/united-st...,2,$ \nOff the Market,"New York\n, United States"
668,\nBlackbook Listing\n(2879) \n,https://www.privateislandsonline.com/united-st...,60 - 70,$ \nInvestment Opportunity\n,"Florida\n, United States"
669,\nBlackbook Listing\n(2926) \n,https://www.privateislandsonline.com/united-st...,50 - 100,$ \nPrice Upon Request\n,"Maine\n, United States"


## Pulizia dataset

In [183]:
df['Name'] = df['Name'].str.replace('\n', '')
df['Price in USD'] = df['Price in USD'].str.replace('\n', '')
df['Price in USD'] = df['Price in USD'].str.replace('$', '')
df['Price in USD'] = df['Price in USD'].str.replace('USD', '')
df['Price in USD'] = df['Price in USD'].str.replace(',', '')
df['Location'] = df['Location'].str.replace('\n', '')

<ipython-input-183-eb95bf02a758>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Price in USD'] = df['Price in USD'].str.replace('$', '')


In [190]:
df["Max sizes in acre"] = df['Size in acres'].str.split(pat="-").str[1]

In [192]:
df['Size in acres'] = df['Size in acres'].str.split(pat="-").str[0]

In [195]:
df["Regione"] = df['Location'].str.split(pat=",").str[-1]

In [196]:
df

,Name,Url,Size in acres,Price in USD,Location,Max sizes in acre,Regione
0,Rangyai Island,https://www.privateislandsonline.com/asia/thai...,110,160000000.00,"Thailand, Asia",NaN,Asia
1,Blackbook Listing(2943),https://www.privateislandsonline.com/africa/se...,170,Price Upon Request,"Seychelles, Africa",190,Africa
2,Blackbook Listing(2867),https://www.privateislandsonline.com/europe/sp...,100,Price Upon Request,"Spain, Europe",200,Europe
3,Pumpkin Key,https://www.privateislandsonline.com/united-st...,26,95000000.00,"Florida, United States",NaN,United States
4,Spectabilis Island,https://www.privateislandsonline.com/caribbean...,460,Sold,"The Exumas, Bahamas, Caribbean",NaN,Caribbean
...,...,...,...,...,...,...,...
666,Savage Island Plantation,https://www.privateislandsonline.com/united-st...,415,Off the Market,"South Carolina, United States",NaN,United States
667,Comfort Island,https://www.privateislandsonline.com/united-st...,2,Off the Market,"New York, United States",NaN,United States
668,Blackbook Listing(2879),https://www.privateislandsonline.com/united-st...,60,Investment Opportunity,"Florida, United States",70,United States
669,Blackbook Listing(2926),https://www.privateislandsonline.com/united-st...,50,Price Upon Request,"Maine, United States",100,United States


In [197]:
df.to_csv('isole.csv')

## Altre informazioni utili dai singoli URL

In [130]:
for url in urls:
    req = requests.get(url)
    soup2 = BeautifulSoup(req.content, 'html.parser')
    r_infos = soup2.find("ul", class_="basic-info mb-50")
    infos.append(r_infos.text)
    print("Done")

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


KeyboardInterrupt: 

In [131]:
infos

['NameRangyai IslandRegionThailand\n, AsiaLocationEast of PhuketDevelopmentDevelopedTitleFreeholdTypePrivate IslandPrice$ 160,000,000USDEURGBPAUDCADCZKDKKHKDHUFILSJPYMXNNOKNZDPHPPLNSGDSEKCHFTWDTHBUAHISKHRKRONBGNTRYCLPZARBRLMYRRUBIDRINRKRWCNYSize110.00 AcresLifestylesIslands With Beaches, Large Acreage, Ocean Island, Income Potential',
 'NameBlackbook Listing (ID 2943)RegionSeychelles\n, AfricaDevelopmentNon DevelopedTitleLeaseholdTypePrivate IslandPrice$139,446,675.59\n- $150,602,409.64USDEURGBPAUDCADCZKDKKHKDHUFILSJPYMXNNOKNZDPHPPLNSGDSEKCHFTWDTHBUAHISKHRKRONBGNTRYCLPZARBRLMYRRUBIDRINRKRWCNYSize170 - 190 AcresLifestylesIslands With Beaches, Large Acreage, Ocean Island',
 'NameBlackbook Listing (ID 2867)RegionSpain\n, EuropeDevelopmentDevelopedTitleFreeholdTypePrivate IslandPrice$100,000,000\n- $200,000,000USDEURGBPAUDCADCZKDKKHKDHUFILSJPYMXNNOKNZDPHPPLNSGDSEKCHFTWDTHBUAHISKHRKRONBGNTRYCLPZARBRLMYRRUBIDRINRKRWCNYSize100 - 200 AcresLifestylesLarge Acreage, Ocean Island, Turn Key, Income